
# INSTALLATION

In [2]:
!pip install aif360
!pip install fairlearn

     |████████████████████████████████| 175 kB 8.7 MB/s 
     |████████████████████████████████| 356 kB 15.1 MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30189 sha256=53c23d91e4f52e8e12d2cc3638ef833c57d4d5ce27f426b0bd2df6ac50a09ec7
  Stored in directory: /root/.cache/pip/wheels/56/19/d5/8cad06661aec65a04a0d6785b1a5ad035cb645b1772a4a0882
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491643 sha256=240e85e90685e941364ffdb8b0c94f8c1cf551193e9fa1588f70e41355293086
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built memory-profiler shap
     |████████████████████████████████| 177 kB 7.1 MB/s 


In [3]:
!apt-get install -jre
!java -version

E: Command line option 'j' [from -jre] is not understood in combination with the other options.
openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [4]:
!pip install h2o

     |████████████████████████████████| 164.8 MB 2.8 kB/s 
  Created wheel for h2o: filename=h2o-3.32.1.5-py2.py3-none-any.whl size=164886106 sha256=aef369384a8ab7e0dede569e0279faf7269fbf0ea1f128016e0b7e2226fcd06b
  Stored in directory: /root/.cache/pip/wheels/2f/f4/f6/7115a720596f0b6c377b3d82c28242585c7bb7ab27d430f97c
Successfully built h2o


In [5]:
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 7.7 MB/s 


#IMPORTS

In [6]:
import numpy as np
from mlxtend.feature_selection import  ExhaustiveFeatureSelector
from xgboost import  XGBClassifier
# import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlsxwriter
from openpyxl import  load_workbook

import shap
#suppress setwith copy warning
pd.set_option('mode.chained_assignment',None)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectFwe, SelectPercentile,SelectFdr, SelectFpr, SelectFromModel
from sklearn.feature_selection import chi2, mutual_info_classif
# from skfeature.function.similarity_based import fisher_score
import aif360
import matplotlib.pyplot as plt
from aif360.metrics.classification_metric import ClassificationMetric

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import StandardDataset , BinaryLabelDataset
from sklearn.preprocessing import MinMaxScaler 
MM= MinMaxScaler()
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

import sys
sys.path.append("../")
import os


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpmav1m441
  JVM stdout: /tmp/tmpmav1m441/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpmav1m441/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.5
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_unknownUser_q5npnj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


#**************************LOADING DATASET*******************************

In [9]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'], axis=1)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'], axis=1)

  # normalization of train and test sets
  Fitter= MM.fit(train)
  transformed_train=Fitter.transform(train)
  train=pd.DataFrame(transformed_train, columns= train.columns)

  #test normalization
  transformed_test=Fitter.transform(test)
  test=pd.DataFrame(transformed_test, columns= test.columns)

  # *************CHECKING FAIRNESS IN DATASET**************************
  ## ****************CONVERTING TO BLD FORMAT******************************


  #Transforming the Train and Test Set to BinaryLabel
  advantagedGroup= [{'HOME_LANGUAGE':1}]
  disadvantagedGroup= [{'HOME_LANGUAGE':0}]

  # class Train(StandardDataset):
  #     def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
  #                  favorable_classes= [1],protected_attribute_names=['age'],   privileged_classes=[[1]], ):


  #         super(Train, self).__init__(df=train  , label_name=label_name ,
  #             favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
  #             privileged_classes=privileged_classes ,
  #            )





  # BLD_Train= Train(protected_attribute_names= ['age'],
  #                        privileged_classes= [[1]])

  class Test(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  ## ********************Checking Bias in Data********************************
  DataBias_Checker = BinaryLabelDatasetMetric(BLD_Test    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)

  dsp= DataBias_Checker .statistical_parity_difference() 
  dif= DataBias_Checker.consistency()  
  ddi= DataBias_Checker.disparate_impact() 

  print('The Statistical Parity diference is = {diff}'.format(diff=  dsp  ))
  print('Individual Fairness is = {IF}'.format( IF=  dif ))
  print('Disparate Impact is = {IF}'.format( IF=   ddi  ))
  # ********************SETTING TO H20 FRAME AND MODEL TRAINING*******************************
  x = list(train.columns)
  y = "PROGRAM_ENROLMENT_STATUS"
  x.remove(y)
  Train=h2o.H2OFrame(train)
  Test= h2o.H2OFrame(test)
  Train[y] = Train[y].asfactor()
  Test[y] = Test[y].asfactor()
  aml = H2OAutoML(max_models=10, nfolds=10, include_algos=['GBM'] , stopping_metric='AUTO') #verbosity='info',,'GBM', 'DRF'
  aml.train(x=x, y=y, training_frame=Train)
  best_model= aml.leader
  # a.model_performance()
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  gbm_Predictions= best_model.predict(Test)
  gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['PROGRAM_ENROLMENT_STATUS']= gbm_Predictions.predict.to_numpy()

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'HOME_LANGUAGE':1}]
  disadvantagedGroup= [{'HOME_LANGUAGE':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])

  # # Workbook= pd.ExcelFile(r'/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  # excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  # OldDF= excelBook.get_sheet_by_name("Dropout")#pd.read_excel(Workbook,sheet_name='Dropout')
  #load workbook
  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  Dropout= excelBook['Dropout']
  data= Dropout.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF,
                                          'DATA_SP':dsp,'DATA_CONS':dif,'DATA_DI':ddi})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Dropout', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

#LOGISTIC REGRESSION

In [13]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'], axis=1)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'], axis=1)

  # normalization of train and test sets
  Fitter= MM.fit(train)
  transformed_train=Fitter.transform(train)
  train=pd.DataFrame(transformed_train, columns= train.columns)

  #test normalization
  transformed_test=Fitter.transform(test)
  test=pd.DataFrame(transformed_test, columns= test.columns)

  # *************CHECKING FAIRNESS IN DATASET**************************
  ## ****************CONVERTING TO BLD FORMAT******************************


  #Transforming the Train and Test Set to BinaryLabel
  advantagedGroup= [{'HOME_LANGUAGE':1}]
  disadvantagedGroup= [{'HOME_LANGUAGE':0}]

  # class Train(StandardDataset):
  #     def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
  #                  favorable_classes= [1],protected_attribute_names=['age'],   privileged_classes=[[1]], ):


  #         super(Train, self).__init__(df=train  , label_name=label_name ,
  #             favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
  #             privileged_classes=privileged_classes ,
  #            )





  # BLD_Train= Train(protected_attribute_names= ['age'],
  #                        privileged_classes= [[1]])

  class Test(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  ## ********************Checking Bias in Data********************************
  DataBias_Checker = BinaryLabelDatasetMetric(BLD_Test    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)

  dsp= DataBias_Checker .statistical_parity_difference() 
  dif= DataBias_Checker.consistency()  
  ddi= DataBias_Checker.disparate_impact() 

  print('The Statistical Parity diference is = {diff}'.format(diff=  dsp  ))
  print('Individual Fairness is = {IF}'.format( IF=  dif ))
  print('Disparate Impact is = {IF}'.format( IF=   ddi  ))
  # ********************SETTING TO H20 FRAME AND MODEL TRAINING*******************************
  x = list(train.columns)
  y = "PROGRAM_ENROLMENT_STATUS"
  x.remove(y)
  Train=h2o.H2OFrame(train)
  Test= h2o.H2OFrame(test)
  Train[y] = Train[y].asfactor()
  Test[y] = Test[y].asfactor()
  LogReg = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0)
  LogReg.train(x=x, y=y, training_frame=Train)

  LogReg_Predictions= LogReg.predict(Test)
  LogReg_Predictions= LogReg_Predictions.as_data_frame()
  # *************************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS**************************************
  predicted_df= test.copy()
  predicted_df['PROGRAM_ENROLMENT_STATUS']= LogReg_Predictions.predict.to_numpy()

  # ***************************COMPUTE DISCRIMINATION********************************

  advantagedGroup= [{'HOME_LANGUAGE':1}]
  disadvantagedGroup= [{'HOME_LANGUAGE':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])

  # # Workbook= pd.ExcelFile(r'/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  # excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  # OldDF= excelBook.get_sheet_by_name("Dropout")#pd.read_excel(Workbook,sheet_name='Dropout')
  #load workbook
  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/LogReg/LR_Results.xlsx')
  Dropout= excelBook['Dropout']
  data= Dropout.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF,
                                          'DATA_SP':dsp,'DATA_CONS':dif,'DATA_DI':ddi})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/LogReg/LR_Results.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Dropout', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

The Statistical Parity diference is = -0.025992990654205572
Individual Fairness is = [0.73381295]
Disparate Impact is = 0.9619006849315069
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [CITIZENSHIP_STATUS]


glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.7194244604316546
The Statistical Parity diference is = -0.19691876750700277
Individual Fairness is = [0.66906475]
Disparate Impact is = 0.6913959613696226
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [CITIZENSHIP_STATUS]


glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.697841726618705
The Statistical Parity diference is = -0.19078947368421056
Individual Fairness is = [0.72608696]
Disparate Impact is = 0.706081081081081
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


...

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.7318840579710145
The Statistical Parity diference is = -0.17968043412722334
Individual Fairness is = [0.68985507]
Disparate Impact is = 0.729213993639255
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [CITIZENSHIP_STATUS]


glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6956521739130435
The Statistical Parity diference is = -0.18475166086681427
Individual Fairness is = [0.67391304]
Disparate Impact is = 0.7081459270364818
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [CITIZENSHIP_STATUS]


glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.7608695652173914


In [14]:
  LogReg = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0)
  LogReg.train(x=x, y=y, training_frame=Train)

  LogReg_Predictions= LogReg.predict(Test)
  LogReg_Predictions= LogReg_Predictions.as_data_frame()
  # *************************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS**************************************
  predicted_df= test.copy()
  predicted_df['PROGRAM_ENROLMENT_STATUS']= LogReg_Predictions.predict.to_numpy()

  # ***************************COMPUTE DISCRIMINATION********************************

Dropping bad and constant columns: [CITIZENSHIP_STATUS]


glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
